In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Carregando os dados de treinamento e teste
train_df = pd.read_csv("train_audience.csv").dropna()  # Carregando dados de treino
train_df = train_df.groupby('Rating', group_keys=False).apply(lambda x: x.sample(1500)).reset_index(drop=True)

vectorizer = TfidfVectorizer(stop_words='english', max_features=100)  # Adjust max_features as needed
reviews = vectorizer.fit_transform(train_df['Review']).toarray()
rating = train_df['Rating']

/tmp/ipykernel_59233/4174741228.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = train_df.groupby('Rating', group_keys=False).apply(lambda x: x.sample(1500)).reset_index(drop=True)


In [ ]:
import tensorflow as tf
import random
import numpy as np
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


x_train, x_val, y_train, y_val = train_test_split (reviews, rating,test_size = 0.2)

/home/felipedias/BigData/Trabalho1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-23 20:27:22.208468: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 20:27:22.215173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732404442.223748   59233 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732404442.226249   59233 cuda_bla

In [3]:
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns


def evaluate_metrics(y, p):
    # Cálculo das métricas de acerto.
    print('MAE (Mean Abs Error):', round(metrics.mean_absolute_error(y, p),3))
    print('MSE (Mean Squared Error)', round(metrics.mean_squared_error(y, p), 3))
    print('Variance', round(metrics.explained_variance_score(y,p),3))

In [4]:
from sklearn.ensemble import RandomForestRegressor

rndForest = RandomForestRegressor(max_depth=8, n_estimators=250, max_features=100)
rndForest.fit(x_train, y_train)
p_val = rndForest.predict(x_val)

In [5]:
evaluate_metrics(y_val, p_val)

MAE (Mean Abs Error): 1.128
MSE (Mean Squared Error) 1.768
Variance 0.146
